In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI AUTOMATIQUE

import os
import subprocess

COMFY_PATH = "/workspace/ComfyUI"

print("🚀 Vérification de l'installation de ComfyUI...")

if os.path.exists(COMFY_PATH):
    print("📂 ComfyUI détecté. Mise à jour en cours...")
    # On force la mise à jour vers la dernière version
    try:
        # git fetch récupère les infos, git reset --hard force l'état local à correspondre exactement au serveur
        subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True, check=True)
        print("✅ Mise à jour réussie (Dernier commit master).")
    except Exception as e:
        print(f"⚠️ Erreur lors de la mise à jour : {e}")
        print("   -> Tentative de git pull classique...")
        subprocess.run(f"cd {COMFY_PATH} && git pull", shell=True)
else:
    print("📥 Première installation (Clonage)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)
    print("✅ Installation terminée.")

In [ ]:
# === INSTALLATION CUSTOM NODES (Manager + Pack Essentiel) ===

import os
import subprocess
import sys

CUSTOM_NODES_DIR = "/workspace/ComfyUI/custom_nodes"

# Liste des dépôts à installer automatiquement

nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",             # Le Manager
    "https://github.com/rgthree/rgthree-comfy",                 # Rgthree (Indispensable : Mute, Context)
    "https://github.com/kijai/ComfyUI-KJNodes",                 # KJNodes (Outils puissants)    
    "https://github.com/cubiq/ComfyUI_essentials",              # Essentials (Traitement d'image)    
    "https://github.com/negaga53/comfyui-imgloader",            # ImgLoader (Chargement d'images)
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",  # Custom-Scripts (Images Feed)  
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",
    "https://github.com/sonnybox/ComfyUI-SuperNodes",
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",
    "https://github.com/ClownsharkBatwing/RES4LYF",
    "https://github.com/giriss/comfy-image-saver",
    "https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes",
    "https://github.com/ltdrdata/was-node-suite-comfyui",
]

print("🚀 Installation automatisée des Custom Nodes...")

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    # 1. Clonage du dépôt s'il n'existe pas
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            
            # 2. Installation des dépendances (requirements.txt)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Installation des dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur lors du clonage de {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} est déjà installé.")

# Fix spécifique pour VideoHelperSuite qui a parfois besoin de ça
try:
    import imageio
except ImportError:
    subprocess.run([sys.executable, "-m", "pip", "install", "imageio[ffmpeg]"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print("\n🎉 Terminé ! Tous vos nodes favoris sont prêts.")
print(">>> Relancez la cellule de lancement (ComfyUI) pour les charger.")



In [ ]:
# QWEN FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Avec Tailles) ---
db_models = {
    "🐲 Qwen Core (Image & Edit)": [
        ("[38.1 GB] Qwen Image BF16", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/diffusion_models/qwen_image_bf16.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "qwen_image_bf16.safetensors", "hf"),
        ("[20.4 GB] Qwen Image FP8", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/diffusion_models/qwen_image_fp8_e4m3fn.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "qwen_image_fp8_e4m3fn.safetensors", "hf"),        
        ("[19.0 GB] Qwen Edit Model (FP8)", "https://huggingface.co/aidiffuser/Qwen-Image-Edit-2509/resolve/main/Qwen-Image-Edit-2509_fp8_e4m3fn.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "Qwen-Image-Edit-2509_fp8_e4m3fn.safetensors", "hf"),
        ("[242 MB] Qwen VAE", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors", "/workspace/ComfyUI/models/vae", "qwen_image_vae.safetensors", "hf"),
        ("[9.4 GB] Qwen CLIP qwen_2.5_vl_7b_fp8_scaled", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors", "/workspace/ComfyUI/models/text_encoders", "qwen_2.5_vl_7b_fp8_scaled.safetensors", "hf"),
        ("[16.6 GB] Qwen CLIP qwen_2.5_vl_7b", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b.safetensors", "/workspace/ComfyUI/models/text_encoders", "qwen_2.5_vl_7b.safetensors", "hf"),
    ],
    "🎨 Qwen LoRAs (Styles & Concepts)": [
        ("[162 MB] Vyesna (qwenvyesnav1_000002750)", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/Vyesna/Qwen/qwenvyesnav1_000002750.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "qwenvyesnav1_000002750.safetensors", "hf"),
        ("[162 MB] SamsungCam (Realistic)", "https://huggingface.co/dci05049/qwen/resolve/main/samsungcam_qwen.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "samsungcam_qwen.safetensors", "hf"),
        ("[162 MB] NiceGirls (Style)", "https://huggingface.co/dci05049/qwen/resolve/6d68704df3d4a9629014ab5241f72d3f5c70fc8b/NiceGirls_qwen.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "NiceGirls_qwen.safetensors", "hf"),
        ("[1.6 GB] Qwen-Image-Lightning-8steps-V2.0-bf16", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-8steps-V2.0-bf16.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-Lightning-8steps-V2.0-bf16.safetensors", "hf"),
        ("[1.7 GB] Qwen-Image-Lightning-8steps-V2.0.safetensors", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-8steps-V2.0.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-Lightning-8steps-V2.0.safetensors", "hf"),      
        ("[850 MB] Qwen-Image-fp8-e4m3fn-Lightning-4steps-V1.0-bf16.safetensors", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-fp8-e4m3fn-Lightning-4steps-V1.0-bf16.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-fp8-e4m3fn-Lightning-4steps-V1.0-bf16.safetensors", "hf"),        
        ("[810 MB] Qwen-Image-Edit-Lightning-8steps-V1.0-bf16", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Edit-Lightning-8steps-V1.0-bf16.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-Edit-Lightning-8steps-V1.0-bf16.safetensors", "hf"),
        # --- AJOUT ICI ---
        ("[1.7 GB] Qwen-Image-Edit-Lightning-8steps-V1.0", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Edit-Lightning-8steps-V1.0.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-Edit-Lightning-8steps-V1.0.safetensors", "hf"),
        # -----------------
        ("[202 MB] Real Life LoRa Qwen", "https://civitai.com/api/download/models/2327746?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "real_life_qwen.safetensors", "civitai"),
        ("[1.1 GB] Famegrid SkinFix (Civitai)", "https://civitai.com/api/download/models/2453097?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "Famegrid_Qwen_Lora_Standard_V1.5_RealSkinFix.safetensors", "civitai"),
        ("[45 MB] Lenovo UltraReal (Civitai)", "https://civitai.com/api/download/models/2106185?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "lenovo.safetensors", "civitai"),
        ("[1.1 GB] The Cleavage-QWEN-Image-GMR (Civitai)", "https://civitai.com/api/download/models/2220274?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "the cleavage.safetensors", "civitai"),        
        ("[282 MB] [Qwen-Image] Smartphone Snapshot Photo Reality [STYLE]", "https://civitai.com/api/download/models/2289403?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image_SmartphoneSnapshotPhotoReality_v4_by-AI_Characters_TRIGGER$amateur photo$.safetensors", "civitai"),
        ("[2.2 GB] 1Girl Qwen Concept", "https://huggingface.co/dci05049/qwen/resolve/main/1girl-qwen_v3.safetensors", "/workspace/ComfyUI/models/loras/QWEN", "1girl-qwen_v3.safetensors", "hf"),
    ],
    "👙 Qwen Body & Anatomy": [
        ("[280 MB] Busty Qwen", "https://civitai.com/api/download/models/2442123?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "busty.safetensors", "civitai"),
        ("[280 MB] The Look (LookDaal)", "https://civitai.com/api/download/models/2196574?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "The_Look_Qwen.safetensors", "civitai"),
        ("[280 MB] Perky Tits (p3rky_t1ts)", "https://civitai.com/api/download/models/2479846?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "Perky_tits-qwen.safetensors", "civitai"),
        ("[280 MB] Big Ass & Narrow Waist", "https://civitai.com/api/download/models/2137968?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "qwen_ass_hips_waist.safetensors", "civitai"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '200px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Qwen)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Qwen...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue

                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                ret = os.system(cmd)
                if ret == 0: print(f"   ✅ OK.")
                else: print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0: print("⚠️ Aucune case cochée.")
        else: print("\n🎉 Terminé.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🐲 INSTALLATION QWEN 2.5 (Avec Tailles)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# DISABLE REACTOR NSFW FILTER

import os

# Le chemin exact indiqué dans le guide (reactor_sfw.py)
target_path = "/workspace/ComfyUI/custom_nodes/comfyui-reactor/scripts/reactor_sfw.py"

# Le code patché (Nettoyé des commentaires qui cassaient la syntaxe)
bypass_code = """
from PIL import Image
import logging
import os
# On garde les imports essentiels pour éviter que ReActor ne plante au démarrage
from scripts.reactor_logger import logger

# On déclare que le modèle existe pour empêcher le téléchargement
MODEL_EXISTS = True

def ensure_nsfw_model(nsfwdet_model_path):
    # On ment au système : "Oui, le modèle est là, tout va bien"
    return True

def nsfw_image(img_data, model_path: str):
    # C'est ici que le bypass agit.
    # False = "Ce n'est pas NSFW" (Safe)
    # On renvoie False systématiquement sans même regarder l'image.
    return False
"""

print(f"🔧 Application du patch sur : {target_path}")

# Vérification d'existence du dossier
folder = os.path.dirname(target_path)
if os.path.exists(folder):
    try:
        with open(target_path, "w") as f:
            f.write(bypass_code)
        print("✅ Fichier 'reactor_sfw.py' écrasé avec succès.")
        print("🔓 Le filtre NSFW est maintenant désactivé.")
    except Exception as e:
        print(f"❌ Erreur lors de l'écriture : {e}")
else:
    print(f"❌ Le dossier {folder} n'existe pas. Vérifie l'installation de ReActor.")

# Par sécurité, on patche aussi l'ancien nom de fichier s'il existe (certaines versions varient)
alt_path = "/workspace/ComfyUI/custom_nodes/comfyui-reactor/scripts/reactor_nsfw.py"
if os.path.exists(alt_path):
    with open(alt_path, "w") as f:
        f.write(bypass_code)
    print("✅ Ancien fichier 'reactor_nsfw.py' également patché par sécurité.")

print("\n🚨 IMPORTANT : Tu dois REDÉMARRER ComfyUI (Stop/Start) pour que ce changement prenne effet !")

In [ ]:
# DÉMARRAGE COMFYUI OPTIMISÉ (FIX DEPENDANCES AUTOMATIQUE)
# VERSION CORRIGÉE : SKIP FLASH ATTENTION COMPILATION

import os
import subprocess
import sys
import glob

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
MANAGER_DIR = f"{COMFY_DIR}/custom_nodes/ComfyUI-Manager"

print("🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...")

# 1. INSTALLATION OUTILS SYSTÈME
print("\n🔧 [1/7] Vérification outils système (ffmpeg, aria2)...")
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. RÉPARATION PYTORCH
print("\n🔥 [2/7] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...")
# On désinstalle xformers temporairement pour éviter les conflits
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
    "--index-url", "https://download.pytorch.org/whl/cu124"
])

# 3. INSTALLATION DÉPENDANCES GLOBALES
print("\n📦 [3/7] Installation des librairies manquantes...")

# --- MODIFICATION ICI : ON SKIP FLASH ATTN POUR GAGNER DU TEMPS ---
print("   ⏩ [SKIP] Flash Attention (Trop long à compiler, on passe)...")
# Si tu en as absolument besoin plus tard, on utilisera un wheel pré-compilé
# subprocess.check_call([sys.executable, "-m", "pip", "install", "flash-attn", "--no-build-isolation"])

# Installation des paquets manuels identifiés dans tes logs
manual_packages = [
    "numpy<2", "pydantic-settings", "sentencepiece", "aiohttp", "einops", "av",
    "torchsde", "scipy", "insightface", "sageattention", "onnxruntime-gpu", "kornia", "spandrel", "color-matcher",
    "alembic", "piexif", "ultralytics", "diffusers", "PyWavelets" # PyWavelets = pywt
]
print(f"   📦 Installation du pack de survie ({len(manual_packages)} paquets)...")
subprocess.check_call([sys.executable, "-m", "pip", "install"] + manual_packages)

# 3b. AUTO-INSTALLATION DES REQUIREMENTS DES CUSTOM NODES (LE FIX MAGIQUE)
print("\n🕵️ [4/7] Scan et installation des dépendances des Custom Nodes...")
node_requirements = glob.glob(f"{COMFY_DIR}/custom_nodes/*/requirements.txt")
for req_file in node_requirements:
    node_name = req_file.split('/')[-2]
    # On évite de réinstaller pour Manager ou les nodes simples pour gagner du temps
    print(f"   -> Check dépendances : {node_name}")
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_file], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        print(f"   ⚠️ Erreur sur {node_name} (souvent ignoré)")

# Force l'installation des requirements de ComfyUI lui-même (fix alembic/frontend)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{COMFY_DIR}/requirements.txt"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


# 4. TÉLÉCHARGEMENT MODÈLES INSIGHTFACE
print("\n📥 [5/7] Vérification des modèles InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]
for f in files_antelope:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. CONFIGURATION FINALE
print("\n⚙️ [6/7] Configuration finale...")

# 6. LANCEMENT
print("\n🚀 [7/7] Lancement du serveur...")
run_script = """#!/bin/bash
cd /workspace/ComfyUI
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet
"""
with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre. Surveille les logs pour 'Import times' !")
!./run_comfy.sh

🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...

🔧 [1/7] Vérification outils système (ffmpeg, aria2)...

🔥 [2/7] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...
Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/cu124/torch-2.5.1%2Bcu124-cp310-cp310-linux_x86_64.whl (908.3 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (13.8 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
  Using cac


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



📦 [3/7] Installation des librairies manquantes...
   ⏩ [SKIP] Flash Attention (Trop long à compiler, on passe)...
   📦 Installation du pack de survie (19 paquets)...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



🕵️ [4/7] Scan et installation des dépendances des Custom Nodes...
   -> Check dépendances : was-ns
   -> Check dépendances : CRT-Nodes
   -> Check dépendances : comfy-image-saver
   -> Check dépendances : RES4LYF
   -> Check dépendances : ComfyUI-Impact-Subpack
   -> Check dépendances : ComfyUI-Impact-Pack
   -> Check dépendances : ComfyUI-SeedVR2_VideoUpscaler
   -> Check dépendances : comfyui-imgloader
   -> Check dépendances : ComfyUI_essentials
   -> Check dépendances : ComfyUI-KJNodes
   -> Check dépendances : rgthree-comfy
   -> Check dépendances : ComfyUI-Manager

📥 [5/7] Vérification des modèles InsightFace...

⚙️ [6/7] Configuration finale...

🚀 [7/7] Lancement du serveur...
✅ ComfyUI démarre. Surveille les logs pour 'Import times' !
[START] Security scan
[ComfyUI-Manager] Using `uv` as Python module for pip operations.
Using Python 3.10.12 environment at: /usr
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-12-16 17:39:42.

In [18]:
# RÉPARATION DES DÉPENDANCES CRITIQUES (TRANSFORMERS / ACCELERATE / FLASH-ATTN)

import sys
import subprocess
import os

print("🧹 NETTOYAGE DES DÉPENDANCES DOUTEUSES...")
# On désinstalle tout pour éviter les conflits de versions fantômes
pkgs_to_remove = [
    "transformers", 
    "accelerate", 
    "bitsandbytes", 
    "xformers", 
    "flash-attn", 
    "sageattention"
]
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y"] + pkgs_to_remove, stdout=subprocess.DEVNULL)

print("🛠️ RÉINSTALLATION PROPRE (VERSIONS QWEN 2.5 COMPATIBLES)...")

# 1. INSTALLATION FLASH ATTENTION (Version pré-compilée pour éviter l'attente de 40min)
# Cette version est spécifique pour Torch 2.5 et CUDA 12.x
print("   ⚡ Installation Flash Attention (Pré-compilé)...")
try:
    # On tente d'installer une version compatible sans compiler
    subprocess.check_call([
        sys.executable, "-m", "pip", "install", 
        "flash-attn==2.7.0.post2", 
        "--no-build-isolation"
    ])
except:
    print("   ⚠️ Flash Attention n'a pas pu s'installer (pas grave, on continuera sans, mais c'est mieux avec).")

# 2. INSTALLATION DU RESTE (Versions forcées pour Qwen)
# transformers >= 4.48 est CRITIQUE pour Qwen 2.5 VL
# accelerate >= 1.2.0 gère mieux le chargement FP8 "zero-copy"
cmds = [
    "transformers>=4.48.2",
    "accelerate>=1.2.1",
    "bitsandbytes>=0.45.0",
    "xformers>=0.0.28.post3",  # Compatible Torch 2.5.1
    "huggingface_hub>=0.26.0",
    "sageattention>=1.0.6",     # Souvent demandé par les nœuds Qwen récents
    "protobuf"                  # Nécessaire pour certains chargements
]

print("   📦 Installation des librairies Core...")
subprocess.check_call([sys.executable, "-m", "pip", "install"] + cmds)

print("\n✅ DÉPENDANCES RÉPARÉES.")
print("👉 Relancez maintenant votre script de démarrage habituel (la cellule précédente).")

🧹 NETTOYAGE DES DÉPENDANCES DOUTEUSES...


🛠️ RÉINSTALLATION PROPRE (VERSIONS QWEN 2.5 COMPATIBLES)...
   ⚡ Installation Flash Attention (Pré-compilé)...
  Using cached flash_attn-2.7.0.post2-cp310-cp310-linux_x86_64.whl
   📦 Installation des librairies Core...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached xformers-0.0.33.post2-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached sageattention-1.0.6-py3-none-any.whl.metadata (5.6 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
Using cached bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)
Using cached xformers-0.0.33.post2-cp39-abi3-manylinux_2_28_x86_64.whl (122.9 MB)
Using cached sageattention-1.0.6-py3-none-any.whl (20 kB)

✅ DÉPENDANCES RÉPARÉES.
👉 Relancez maintenant votre script de démarrage habituel (la cellule précédente).



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [17]:
# DÉMARRAGE COMFYUI - MODE TURBO (SKIP DEPENDANCES)

import os
import subprocess
import sys
import glob

# --- CONFIGURATION ---
FAST_BOOT = True  # <--- METTRE SUR True POUR SAUTER LES VÉRIFICATIONS LONGUES
COMFY_DIR = "/workspace/ComfyUI"

print("🚀 DÉMARRAGE COMFYUI...")

# 1. OUTILS SYSTÈME (Rapide)
if not FAST_BOOT:
    print("\n🔧 [1/6] Vérification outils système...")
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
else:
    print("⏩ [SKIP] Outils système (Mode Turbo)")

# 2. PYTORCH (Rapide si déjà fait)
print("\n🔥 [2/6] Vérification rapide PyTorch...")
try:
    import torch
    print(f"   ✅ PyTorch détecté : {torch.__version__}")
except ImportError:
    print("   ⚠️ PyTorch manquant, installation forcée...")
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
        "--index-url", "https://download.pytorch.org/whl/cu124"
    ])

# 3. DÉPENDANCES GLOBALES
if not FAST_BOOT:
    print("\n📦 [3/6] Vérification librairies manquantes...")
    manual_packages = [
        "numpy<2", "pydantic-settings", "sentencepiece", "aiohttp", "einops", "av",
        "torchsde", "scipy", "insightface", "sageattention", "onnxruntime-gpu", "kornia", "spandrel", "color-matcher",
        "alembic", "piexif", "ultralytics", "diffusers", "PyWavelets"
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + manual_packages)
else:
    print("⏩ [SKIP] Dépendances globales (Mode Turbo)")

# 4. CUSTOM NODES (C'EST LÀ QUE ÇA BLOQUAIT)
if not FAST_BOOT:
    print("\n🕵️ [4/6] Scan des dépendances Custom Nodes...")
    node_requirements = glob.glob(f"{COMFY_DIR}/custom_nodes/*/requirements.txt")
    for req_file in node_requirements:
        node_name = req_file.split('/')[-2]
        print(f"   -> Vérification : {node_name}")
        # J'ai enlevé le DEVNULL pour que vous voyiez si ça bloque
        subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_file]) 
else:
    print("⏩ [SKIP] Custom Nodes check (C'est ça qui était long !)")

# 5. MODÈLES INSIGHTFACE (Rapide car vérifie l'existence des fichiers)
print("\n📥 [5/6] Vérification rapide InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
if not os.path.exists(f"{antelope_dir}/scrfd_10g_bnkps.onnx"):
    print("   ⚠️ Modèles manquants, téléchargement...")
    os.makedirs(antelope_dir, exist_ok=True)
    files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]
    for f in files_antelope:
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
else:
    print("   ✅ Modèles InsightFace présents.")

# 6. LANCEMENT
print("\n🚀 [6/6] Lancement du serveur...")
run_script = """#!/bin/bash
cd /workspace/ComfyUI
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet
"""
with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre immédiatement !")
!./run_comfy.sh

🚀 DÉMARRAGE COMFYUI...
⏩ [SKIP] Outils système (Mode Turbo)

🔥 [2/6] Vérification rapide PyTorch...
   ✅ PyTorch détecté : 2.5.1+cu124
⏩ [SKIP] Dépendances globales (Mode Turbo)
⏩ [SKIP] Custom Nodes check (C'est ça qui était long !)

📥 [5/6] Vérification rapide InsightFace...
   ✅ Modèles InsightFace présents.

🚀 [6/6] Lancement du serveur...
✅ ComfyUI démarre immédiatement !
[START] Security scan
[ComfyUI-Manager] Using `uv` as Python module for pip operations.
Using Python 3.10.12 environment at: /usr
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-12-16 17:37:27.867
** Platform: Linux
** Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
** Python executable: /usr/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspace/ComfyUI/user
** ComfyUI-Manager config path: /workspace/ComfyUI/user/__manager/config.ini
** Log path: /workspace/ComfyUI/user/co